# LaFeAsO

In this notebook we further analyze the muon sites in LaFeAsO.

More precisley we will consider:

1. the equivalent positions for the lowest energy site
2. inspect the expected local field at these sites

TODOs are highlighted by 🖍️, check instructions and try yourself!

In [1]:
# First we import relevant modules

from pymatgen.core import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
import numpy as np
from weas_widget import WeasWidget

In [2]:
# This function add atoms in equivalent positions for a given spacegroup
# you don't need to study it, but it's convenient to have in the scripts!
def add_atom_via_asymmetric_unit(
    structure,
    frac_coords,
    species,
    symprec=1e-3
):
    """
    Add an atom by rebuilding the structure from the asymmetric unit
    plus the new atom, using Structure.from_spacegroup.

    Parameters
    ----------
    structure : pymatgen.core.Structure
        Input structure.
    frac_coords : array-like
        Fractional coordinates of the new atom.
    species : str or Element
        Species of the added atom.
    symprec : float
        Symmetry tolerance.

    Returns
    -------
    pymatgen.core.Structure
        Fully symmetrized structure including the new atom.
    """

    sga = SpacegroupAnalyzer(structure, symprec=symprec)
    symm_struct = sga.get_symmetrized_structure()

    spacegroup = sga.get_space_group_symbol()
    lattice = structure.lattice

    # Extract symmetry-inequivalent sites
    species_list = []
    coords_list = []

    for eq_sites in symm_struct.equivalent_sites:
        site = eq_sites[0]
        species_list.append(site.specie)
        coords_list.append(site.frac_coords)

    # Add new atom generator
    species_list.append(species)
    coords_list.append(np.array(frac_coords))

    # Rebuild full structure
    new_structure = Structure.from_spacegroup(
        spacegroup,
        lattice=lattice,
        species=species_list,
        coords=coords_list
    )

    return new_structure

## Summary table

Let's first a second look again at the summary table:

In [3]:
import pandas as pd

df = pd.read_csv("Summary_table.csv")
display(df[['label', 'delta_E', 'muon_position_cc' , 'B_T']])

,label,delta_E,muon_position_cc,B_T
0,A,0,"[0.052, 0.127, 0.208]","[-1.3, 0.002, 0.181]"
1,B,0,"[-0.0, 0.055, 0.204]","[0.0, -0.151, 0.306]"
2,C,188,"[0.125, 0.0, 0.154]","[-0.907, 0.002, -0.001]"
3,D,443,"[0.0, -0.123, 0.096]","[-0.0, -0.0, 0.048]"
4,E,601,"[0.022, -0.0, 0.0]","[0.0, -0.0, 0.016]"


There are many intersting points to notice in this table:
1. many sites have very similar energies and local fields. Although the algorithm does the best it can to find equivalent position we may doubt: are all sites really different?
2. Some coordindates are close to high symmetry points (which one?)
3. Some sites have very large energy!

## Inspect the supercell

The picture below shows the supercell that was computed.

You can show bonds between atoms by changing the search lenght of the function `add_bond_pair`.

Change it so that the F-mu-F site shows up clearly.

In [4]:
viewer = WeasWidget()
viewer.from_pymatgen(Structure.from_file('Supercell_A.cif'))
viewer.avr.model_style = 1
viewer.avr.bond.add_bond_pair('Fe', 'H', max=3.)  # 🖍️ change `max` in order to show Fe-H bond
viewer

WeasWidget(children=(<weas_widget.base_widget.BaseWidget object at 0x7fed8d2b42f0>,))

## Equivalent positions

We obtained an equilibrium position in a supercell. While clearly the muon reduces the symmetry of our lattice, it is usefull to show its position in the initial undistorted cell.

To do so we need to find a mapping between our supercell and the original cell. But these's a problem, **our supercell is distorted by the presence of the muon**.

We'll need a bit of trial and error.

### Finding the original spacegroup

Let's start with the analysis of the unit cell. You can go back to the presentation and check the spacegroup there, but you have two other alternatives:

1. use the file `Allsites_unitcell.cif` that already reports the positions in the undistorted unit cell. 
    You can remove the muons from there and find the spacegroup with a convenient pymatgen function,
    `get_symmetrized_structure()` provided by `SpacegroupAnalyzer`.
2. use the same approach but on the distorted supercell.

In general both approaches give you valuable information to check that everything is consistent.

Let's do both.


In [5]:
# Method 1

LaFeAsO = Structure.from_file('Allsites_unitcell.cif').remove_species('H')
SpacegroupAnalyzer(LaFeAsO, symprec=0.001).get_symmetrized_structure()

SymmetrizedStructure
Full Formula (La8 Fe8 As8 O8)
Reduced Formula: LaFeAsO
Spacegroup: P4/nmm (129)
abc   :   5.706300   5.678800  17.419000
angles:  90.000000  90.000000  90.000000
Sites (32)
  #  SP       a     b        c  Wyckoff
---  ----  ----  ----  -------  ---------
  0  La    0     0.25  0.071    8c
  1  Fe    0.25  0     0.25     8b
  2  As    0     0.25  0.32525  8c
  3  O     0.25  0     0        8a

🧐 Check the result: What spacegroup has been identified? Does it match with your expectations?

In [6]:
# Method 2

LaFeAsO = Structure.from_file('Supercell_A.cif').remove_species('H')
SpacegroupAnalyzer(LaFeAsO, symprec=0.5).get_symmetrized_structure()  # 🖍️ change the value of `symprec`, what happens? Why?

SymmetrizedStructure
Full Formula (La32 Fe32 As32 O32)
Reduced Formula: LaFeAsO
Spacegroup: P4/nmm (129)
abc   :  11.412600  11.357600  17.419000
angles:  90.000000  90.000000  90.000000
Sites (128)
  #  SP           a         b         c  Wyckoff
---  ----  --------  --------  --------  ---------
  0  La    0.999902  0.125039  0.068654  32c
  1  Fe    0.1208    0.010727  0.246781  32b
  2  As    0.995192  0.125337  0.331941  32c
  3  O     0.125539  0.999704  0.998666  32a

🧐 Check the result: does it match with your expectations?
If not, why?

**Hint**: try changing how the algorithm checks for equivalent positions. This is governed by `symprec`.

## Mapping positions back to the unit cell

In order to do this we need to know what transformation matrix was used to produce the supercell. This information can be extracted fromt the AiiDA repo, but we find it here just by comparing the initial and the final structures.

To this aim, we will use `StructureMatcher` function of pymatgen.

In [7]:
from pymatgen.analysis.structure_matcher import StructureMatcher

supercell_A = Structure.from_file('Supercell_A.cif')
supercell_A_without_muon = supercell_A.copy().remove_species('H')


LaFeAsO = Structure.from_file('Allsites_unitcell.cif').remove_species('H')
SpacegroupAnalyzer(LaFeAsO, symprec=0.1).get_conventional_standard_structure().to('LaFeAsO_unitcell.cif')


LaFeAsO_unitcell = Structure.from_file('LaFeAsO_unitcell.cif')

# StructureMatcher can accept different tolerances for judging equivalence
matcher = StructureMatcher(primitive_cell=False, attempt_supercell=True)

# first, we can verify these lattices are equivalent
assert(matcher.fit(supercell_A_without_muon,LaFeAsO_unitcell))

# and we can get the transformation matrix from one to the other
# this returns the supercell matrix (e.g. change of basis),
# as well as any relevant translation, and mapping of atoms from one
# crystal to the other
M, T, amap = matcher.get_transformation(supercell_A_without_muon,LaFeAsO_unitcell)

print('Transformation matrix is\n', M)
print('Translation vector is\n', T)

Transformation matrix is
 [[ 2 -2  0]
 [ 2  2  0]
 [ 0  0 -2]]
Translation vector is
 [-3.75404407e-01  4.99991775e-01 -3.12903736e-04]
Transformation matrix is
 [[ 2 -2  0]
 [ 2  2  0]
 [ 0  0 -2]]
Translation vector is
 [-3.75404407e-01  4.99991775e-01 -3.12903736e-04]


In [8]:
Hidx = supercell_A.indices_from_symbol('H')

H_frac_coords = supercell_A.frac_coords[Hidx]

H_frac_coords_unit_cell = ((( H_frac_coords - T) @ M) % 1.)

print(H_frac_coords_unit_cell)

[0.10845916 0.39778506 0.5839375 ]
[0.10845916 0.39778506 0.5839375 ]


In [9]:
from weas_widget import WeasWidget


viewer = WeasWidget()

scA = add_atom_via_asymmetric_unit(LaFeAsO_unitcell, H_frac_coords_unit_cell, 'H')
scA.translate_sites(range(len(scA)), [0.5,0.,0])
viewer.from_pymatgen( scA )

viewer.avr.model_style = 1
viewer.avr.bond.add_bond_pair('Fe', 'H', max=2.2)
viewer

WeasWidget(children=(<weas_widget.base_widget.BaseWidget object at 0x7fed8d2a1bd0>,))

## What next

At this stage you could:
1. repeat the same analysis for other sites/supercells, or
2. compute the local fields at the muon site with your favorite tool or with your own code.



# Local fields for equivalent sites

As already pointed out, the equivalent muon sites for supercell A, the lowest energy results, are 4.

In [10]:
LaFeAsO_unitcell = Structure.from_file('LaFeAsO_unitcell.cif')
add_atom_via_asymmetric_unit(LaFeAsO_unitcell, [xx, yy, zz], 'H') # 🖍️ replace xx, yy, zz with the coordinares of the lowest energy site. 
# Also try approximating positions to 0.5, how many positions do you get?

Structure Summary
Lattice
    abc : 4.02525245 4.02525245 8.7095
 angles : 90.0 90.0 90.0
 volume : 141.1170436344289
      A : np.float64(4.02525245) np.float64(0.0) np.float64(2.464756264326271e-16)
      B : np.float64(6.473099635511372e-16) np.float64(4.02525245) np.float64(2.464756264326271e-16)
      C : np.float64(0.0) np.float64(0.0) np.float64(8.7095)
    pbc : True True True
PeriodicSite: La (3.237e-16, 2.013, 7.473) [0.0, 0.5, 0.858]
PeriodicSite: La (2.013, 0.0, 1.237) [0.5, 0.0, 0.142]
PeriodicSite: Fe (0.0, 0.0, 4.355) [0.0, 0.0, 0.5]
PeriodicSite: Fe (2.013, 2.013, 4.355) [0.5, 0.5, 0.5]
PeriodicSite: As (3.237e-16, 2.013, 3.044) [0.0, 0.5, 0.3495]
PeriodicSite: As (2.013, 0.0, 5.666) [0.5, 0.0, 0.6505]
PeriodicSite: O (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]
PeriodicSite: O (2.013, 2.013, 2.465e-16) [0.5, 0.5, 0.0]
PeriodicSite: H (2.013, 0.0, 3.658) [0.5, 0.0, 0.42]
PeriodicSite: H (3.237e-16, 2.013, 5.052) [0.0, 0.5, 0.58]

### Let's compute the local field at these sites

Manually add the 4 positions below

In [14]:
from muesr.core import Sample
from muesr.engines.clfc import find_largest_sphere, locfield
from muesr.i_o import load_cif

lafeaso = Sample()
load_cif(lafeaso,"./LaFeAsO_unitcell.cif")

# 🖍️ add muon positions, use fractional coordinates (the ones in square brakets reported above) using the syntax below. 
# (if you have problems, solution is given below)
lafeaso.add_muon([0.4, 0.1, 0.42])
lafeaso.add_muon([0.1, 0.4, 0.58])


# magnetic moment of 2.6 muB from https:doi.//org/10.1103/PhysRevB.87.121108  https://doi.org/10.1103/PhysRevB.69.014417
lafeaso.new_mm()
lafeaso.mm.k=np.array([0.5,0.5,0.0])
lafeaso.mm.fc= np.array([[0.0, 0.0, 0.0],[0.0, 0.0, 0.0], # La
                     [0.5, 0.5, 0.0],[-0.5, -0.5, 0.0],  # Fe
                     [0.0, 0.0, 0.0],[0.0, 0.0, 0.0],  # As
                     [0.0, 0.0, 0.0],[0.0, 0.0, 0.0]   # O
                    ], dtype=complex)

# use 100 cells in each direction
n=100
# find the largest sphere contained in this 100x100x100 supercell
radius=find_largest_sphere(lafeaso,[n,n,n])
# compute the local field
r=locfield(lafeaso, 's', [n, n, n] ,radius)

# Finally print the result
for i in range(len(lafeaso.muons)):
    print('Site {}, Bdip = {:.4f} T with the experimental value of Bexp ~= 0.17 T'.format(i, np.linalg.norm(r[i].D,axis=0)))

Site 0, Bdip = 0.2854 T with the experimental value of Bexp ~= 0.17 T
Site 1, Bdip = 0.2854 T with the experimental value of Bexp ~= 0.17 T
